# Working with different datasets
`huracanpy` can load track data from various formats. For testing, there are a few
example files embedded in `huracanpy`

In [4]:
import huracanpy

print(huracanpy.example_csv_file.split("/")[-1])
print(huracanpy.example_TRACK_netcdf_file.split("/")[-1])
print(huracanpy.example_TRACK_file.split("/")[-1])

sample.csv
tr_trs_pos.2day_addT63vor_addmslp_add925wind_add10mwind.tcident.new.nc
tr_trs_pos.2day_addT63vor_addmslp_add925wind_add10mwind.tcident.new


## CSV

A CSV is a useful way of storing track data. If you tracks are stored in csv (including
if they were outputed from TempestExtremes' StitchNodes), you can specify the
`tracker="csv"` argument, or, if your filename ends with *csv*, it will be detected
automatically.

`huracanpy.load` will read most of the CSV file as it is to output as an
`xarray.Dataset`. There can be a few extra modifications
to make sure the output has the variables `track_id`, `time`, `lon`, and `lat`.
For example, in the file used here, the time variable is constructed from
`year`, `month`, `day`, and `hour`.



In [8]:
huracanpy.load(huracanpy.example_csv_file)

<xarray.Dataset>
Dimensions:   (record: 99)
Coordinates:
  * record    (record) int64 0 1 2 3 4 5 6 7 8 9 ... 90 91 92 93 94 95 96 97 98
Data variables: (12/13)
    track_id  (record) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 2 2 2 2 2 2 2 2 2 2 2 2
    year      (record) int64 1980 1980 1980 1980 1980 ... 1980 1980 1980 1980
    month     (record) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    day       (record) int64 6 6 6 7 7 7 7 8 8 8 ... 28 29 29 29 29 30 30 30 30
    hour      (record) int64 6 12 18 0 6 12 18 0 6 12 ... 18 0 6 12 18 0 6 12 18
    i         (record) int64 482 476 476 477 478 475 ... 230 229 230 234 241 249
    ...        ...
    lon       (record) float64 120.5 119.0 119.0 119.2 ... 57.5 58.5 60.25 62.25
    lat       (record) float64 -14.25 -14.75 -15.0 -15.0 ... -39.25 -42.0 -45.5
    slp       (record) float64 9.988e+04 9.981e+04 ... 9.747e+04 9.754e+04
    zs        (record) float64 -10.71 -16.11 -40.21 ... -225.5 -218.5 -211.5
    wind10    (record) float64 14.65 13.99 13.7 17.98 ... 23.19 23.69 23.96 23.4
    time      (record) datetime64[ns] 1980-01-06T06:00:00 ... 1980-01-30T18:0...

## NetCDF

Similar to CSV, NetCDF data can largely be loaded as is. NetCDF has the disadvantage of
not being readable like a CSV, but the advantage that it can better store metadata about
variables.

The only assumption about the NetCDF file, is that it is using the CF convention

http://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_contiguous_ragged_array_representation_of_trajectories

This allows the load function to identify the TRACK_ID and extend it along the data
dimension. Unlike loading CSV data, variables are not currently renamed. In the example
here `track_id` is upper case and the positions are `longitude` and `latitude` because
that is how they are named in the file.

In [7]:
huracanpy.load(huracanpy.example_TRACK_netcdf_file)

<xarray.Dataset>
Dimensions:                 (record: 4580, tracks: 86)
Coordinates: (12/18)
    TRACK_ID                (record) int32 840 840 840 840 ... 24685 24685 24685
    time                    (record) datetime64[ns] ...
    longitude               (record) float32 ...
    latitude                (record) float32 ...
    longitude_1             (record) float32 ...
    latitude_1              (record) float32 ...
    ...                      ...
    longitude_5             (record) float32 ...
    latitude_5              (record) float32 ...
    longitude_6             (record) float32 ...
    latitude_6              (record) float32 ...
    longitude_7             (record) float32 ...
    latitude_7              (record) float32 ...
Dimensions without coordinates: record, tracks
Data variables: (12/19)
    FIRST_PT                (tracks) int32 ...
    index                   (record) int32 ...
    relative_vorticity      (record) float32 ...
    relative_vorticity_850  (record) float32 ...
    relative_vorticity_700  (record) float32 ...
    relative_vorticity_600  (record) float32 ...
    ...                      ...
    longitude_9             (record) float32 ...
    latitude_9              (record) float32 ...
    wind_speed_925          (record) float32 ...
    longitude_10            (record) float32 ...
    latitude_10             (record) float32 ...
    wind_speed_10m          (record) float32 ...
Attributes:
    realm:        atmos
    history:      testing
    Conventions:  CF-1.7
    featureType:  trajectory

## TRACK

Note that TRACK files don't contain the variable names, instead they are usually
described in the filename. Currently `huracanpy.load` doesn't try to infer the variable
names from the filename. Instead, any extra variables will be named feature_n, where
n is between 0 and number of variables minus 1. TRACK also associates extra coordinates
with some of these features, these will be loaded as feature_n_longitude and
feature_n_latitude.

In [9]:
huracanpy.load(huracanpy.example_TRACK_file, tracker="TRACK")

<xarray.Dataset>
Dimensions:              (record: 46)
Dimensions without coordinates: record
Data variables: (12/35)
    lon                  (record) float64 284.1 287.8 292.5 ... 115.8 116.4
    lat                  (record) float64 29.5 29.99 30.88 ... 16.3 17.69 17.81
    vorticity            (record) float64 2.273 4.124 5.978 ... 2.445 1.589
    feature_0_longitude  (record) float64 285.4 287.7 292.7 ... 116.0 116.2
    feature_0_latitude   (record) float64 29.38 30.58 30.14 ... 18.32 17.78
    feature_0            (record) float64 3.938 6.482 8.459 ... 7.45 6.762 6.356
    ...                   ...
    feature_8            (record) float64 16.1 24.5 25.88 ... 16.93 18.84 17.18
    feature_9_longitude  (record) float64 289.7 289.7 290.5 ... 115.9 121.2
    feature_9_latitude   (record) float64 29.65 31.05 33.02 ... 18.13 21.78
    feature_9            (record) float64 11.22 15.69 16.05 ... 13.9 14.0 13.73
    track_id             (record) int64 840 840 840 840 ... 1601 1601 1601 1601
    time                 (record) datetime64[ns] 2022-01-13T18:00:00 ... 2022...

If you want to load the variables by name, then pass a list of variable names to
`huracanpy.load`. The associated longitudes/latitudes are associated to the respective
feature names.

In [12]:
variable_names = [
    *[f"vorticity_{n}hPa" for n in [850, 700, 600, 500, 400, 300, 200]],
    "mslp",
    "vmax_925hPa",
    "vmax_10m",
]
huracanpy.load(
    huracanpy.example_TRACK_file, tracker="TRACK", variable_names=variable_names
)

<xarray.Dataset>
Dimensions:                     (record: 46)
Dimensions without coordinates: record
Data variables: (12/35)
    lon                         (record) float64 284.1 287.8 ... 115.8 116.4
    lat                         (record) float64 29.5 29.99 ... 17.69 17.81
    vorticity                   (record) float64 2.273 4.124 ... 2.445 1.589
    vorticity_850hPa_longitude  (record) float64 285.4 287.7 ... 116.0 116.2
    vorticity_850hPa_latitude   (record) float64 29.38 30.58 ... 18.32 17.78
    vorticity_850hPa            (record) float64 3.938 6.482 ... 6.762 6.356
    ...                          ...
    vmax_925hPa                 (record) float64 16.1 24.5 25.88 ... 18.84 17.18
    vmax_10m_longitude          (record) float64 289.7 289.7 ... 115.9 121.2
    vmax_10m_latitude           (record) float64 29.65 31.05 ... 18.13 21.78
    vmax_10m                    (record) float64 11.22 15.69 ... 14.0 13.73
    track_id                    (record) int64 840 840 840 ... 1601 1601 1601
    time                        (record) datetime64[ns] 2022-01-13T18:00:00 ....

## IBTrACS
`huracanpy` includes a subset of the IBTrACS dataset to use 

In [8]:
# ibtracs_subset is "wmo" or "usa" which correspond to the slp/variables used
huracanpy.load(tracker="ibtracs", ibtracs_subset="wmo", ibtracs_online=False)

/home/lsaffin/Documents/meteorology/programming/huracanpy/huracanpy/_data/ibtracs.py:90: UserWarning: This offline function loads a light version of IBTrACS which is embedded within the package, based on a file produced manually by the developers.
                  It was last updated on the 24nd May 2024, based on the IBTrACS file at that date.
                  It contains only data from 1980 up to the last year with no provisional tracks. All spur tracks were removed. Only 6-hourly time steps were kept.
  warnings.warn(
/home/lsaffin/Documents/meteorology/programming/huracanpy/huracanpy/_data/ibtracs.py:96: UserWarning: You are loading the IBTrACS-WMO subset.                       This dataset contains the positions and intensity reported by the WMO agency responsible for each basin
                      Be aware of the fact that wind and pressure data is provided as they are in IBTrACS,                       which means in particular that wind speeds are in knots and averaged over 

<xarray.Dataset>
Dimensions:   (record: 139416)
Coordinates:
  * record    (record) int64 0 1 2 3 4 5 ... 139411 139412 139413 139414 139415
Data variables:
    track_id  (record) object '1980001S13173' ... '2021349N05108'
    season    (record) int64 1980 1980 1980 1980 1980 ... 2021 2021 2021 2021
    basin     (record) object 'SP' 'SP' 'SP' 'SP' 'SP' ... 'WP' 'WP' 'WP' 'WP'
    time      (record) datetime64[ns] 1980-01-01 ... 2021-12-17T06:00:00
    lon       (record) float64 172.5 172.4 172.5 172.8 ... 103.0 103.0 101.9
    lat       (record) float64 -12.5 -11.91 -11.5 -11.2 ... 4.3 4.2 4.2 4.0
    wind      (record) float64 nan nan nan nan 30.0 nan ... nan nan nan nan nan
    slp       (record) float64 nan nan nan nan 997.0 nan ... nan nan nan nan nan

You can download the full IBTrACS dataset by setting `ibtracs_online=True`. In this case
the subset refers to the official IBTrACS subsets.

`huracanpy` won't load locally saved copies of IBTrACS. We would recommend downloading
once with `ibtracs_online=True` and subsetting then saving a copy as CSV or NetCDF with
`ibtracs.save`. Also note that the NetCDF files provided by IBTrACS are not (currently)
compatible with `huracanpy` because the format is different.

In [31]:
# Not running this code for the documentation since it downloads the file when run
# huracanpy.load(tracker="IBTrACS", subset="ALL", ibtracs_online=True)

/home/lsaffin/Documents/meteorology/programming/huracanpy/huracanpy/_data/_csv.py:32: DtypeWarning: Columns (12,21,61,66,128,143,148) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv(filename, **read_csv_kws)


<xarray.Dataset>
Dimensions:           (record: 716165)
Coordinates:
  * record            (record) int64 0 1 2 3 4 ... 716161 716162 716163 716164
Data variables: (12/163)
    sid               (record) object '1842298N11080' ... '2024147N19089'
    season            (record) int64 1842 1842 1842 1842 ... 2024 2024 2024 2024
    number            (record) int64 1 1 1 1 1 1 1 1 ... 27 27 27 27 27 27 27 27
    basin             (record) object 'NI' 'NI' 'NI' 'NI' ... 'NI' 'NI' 'NI'
    subbasin          (record) object 'BB' 'BB' 'BB' 'BB' ... 'BB' 'BB' 'BB'
    name              (record) object 'NOT_NAMED' 'NOT_NAMED' ... 'REMAL'
    ...                ...
    usa_searad_se     (record) float64 nan nan nan nan nan ... nan nan nan nan
    usa_searad_sw     (record) float64 nan nan nan nan nan ... nan nan nan nan
    usa_searad_nw     (record) float64 nan nan nan nan nan ... nan nan nan nan
    storm_speed       (record) float64 9.0 9.0 9.0 9.0 9.0 ... 8.0 8.0 7.0 7.0
    storm_dir         (record) float64 266.0 267.0 267.0 ... 39.0 40.0 40.0
    time              (record) datetime64[ns] 1842-10-25T03:00:00 ... 2024-05...

## Saving data

In [13]:
tracks = huracanpy.load(huracanpy.example_csv_file)
huracanpy.save(tracks, "saved_data.csv")
huracanpy.save(tracks, "saved_data.nc")

In [18]:
!head -5 saved_data.csv

track_id,year,month,day,hour,i,j,lon,lat,slp,zs,wind10,time
0,1980,1,6,6,482,417,120.5,-14.25,99876.38,-10.7118,14.64815,1980-01-06 06:00:00
0,1980,1,6,12,476,419,119.0,-14.75,99811.0,-16.10522,13.98848,1980-01-06 12:00:00
0,1980,1,6,18,476,420,119.0,-15.0,99536.94,-40.20874,13.69575,1980-01-06 18:00:00
0,1980,1,7,0,477,420,119.25,-15.0,99414.56,-50.43206,17.97812,1980-01-07 00:00:00


In [20]:
!ncdump -h saved_data.nc

netcdf saved_data {
dimensions:
	record = 99 ;
	trajectory = 3 ;
variables:
	int64 record(record) ;
	int64 track_id(trajectory) ;
	int64 year(record) ;
	int64 month(record) ;
	int64 day(record) ;
	int64 hour(record) ;
	int64 i(record) ;
	int64 j(record) ;
	double lon(record) ;
		lon:_FillValue = NaN ;
	double lat(record) ;
		lat:_FillValue = NaN ;
	double slp(record) ;
		slp:_FillValue = NaN ;
	double zs(record) ;
		zs:_FillValue = NaN ;
	double wind10(record) ;
		wind10:_FillValue = NaN ;
	int64 time(record) ;
		time:units = "hours since 1980-01-06 06:00:00" ;
		time:calendar = "proleptic_gregorian" ;
	int64 rowSize(trajectory) ;
		rowSize:sample_dimension = "record" ;
}
